In [1]:
import os
import numpy as np
import pandas as pd
get_ipython().run_line_magic('matplotlib', 'inline')
import IPython.display as ipd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
import sklearn as skl
import sklearn.utils, sklearn.preprocessing, sklearn.decomposition

import matplotlib.patches as mpatches
from mpl_toolkits.mplot3d import Axes3D
from pandas.plotting import scatter_matrix

import random
from tqdm import tqdm # Progress Bar

from sklearn.preprocessing import LabelEncoder

import warnings
warnings.filterwarnings('ignore')
%matplotlib inline

### Read information of tracks

In [2]:
# read in tracks data
tracks = pd.read_csv("../Data/tracks.csv")
tracks.shape

(106576, 53)

In [3]:
# reformatting tracks data and removes duplicates
tracks = tracks.drop(1)
labels = tracks.iloc[0].values
labels[0] = "track_id"
tracks.columns = labels
print(labels)
tracks = tracks.drop(0)
tracks = tracks.drop_duplicates()

['track_id' 'comments' 'date_created' 'date_released' 'engineer'
 'favorites' 'id' 'information' 'listens' 'producer' 'tags' 'title'
 'tracks' 'type' 'active_year_begin' 'active_year_end' 'associated_labels'
 'bio' 'comments' 'date_created' 'favorites' 'id' 'latitude' 'location'
 'longitude' 'members' 'name' 'related_projects' 'tags' 'website'
 'wikipedia_page' 'split' 'subset' 'bit_rate' 'comments' 'composer'
 'date_created' 'date_recorded' 'duration' 'favorites' 'genre_top'
 'genres' 'genres_all' 'information' 'interest' 'language_code' 'license'
 'listens' 'lyricist' 'number' 'publisher' 'tags' 'title']


Tracks may be assigned to multiple genres or no genre at all.

In [4]:
tracks.iloc[5]["genres_all"]

'[17, 10, 76, 103]'

In [5]:
# keeping only columns with useful features
tracks = tracks[['track_id', 'title', 'location', 'name', 'bit_rate', 'genres_all']].copy()
tracks.head(20)

,track_id,title,title,location,name,bit_rate,genres_all
2,2,AWOL - A Way Of Life,Food,New Jersey,AWOL,256000,[21]
3,3,AWOL - A Way Of Life,Electric Ave,New Jersey,AWOL,256000,[21]
4,5,AWOL - A Way Of Life,This World,New Jersey,AWOL,256000,[21]
5,10,Constant Hitmaker,Freeway,NaN,Kurt Vile,192000,[10]
6,20,Niris,Spiritual Level,Colchester England,Nicky Cook,256000,"[17, 10, 76, 103]"
7,26,Niris,Where is your Love?,Colchester England,Nicky Cook,256000,"[17, 10, 76, 103]"
8,30,Niris,Too Happy,Colchester England,Nicky Cook,256000,"[17, 10, 76, 103]"
9,46,Niris,Yosemite,Colchester England,Nicky Cook,256000,"[17, 10, 76, 103]"
10,48,Niris,Light of Light,Colchester England,Nicky Cook,256000,"[17, 10, 76, 103]"
11,134,AWOL - A Way Of Life,Street Music,New Jersey,AWOL,256000,[21]


In [6]:
# removing instances that do not have a value for genres_all
tracks.drop(tracks.index[tracks["genres_all"].isnull()], inplace=True)
tracks.drop(tracks.index[tracks["genres_all"].equals("['']")], inplace=True)
tracks.head(10)
tracks.shape

(106574, 7)

In [7]:
#get list of track ids to help group sync up
track_ids = tracks.iloc[:]["track_id"].values

In [8]:
MyFile=open('../Data/tracks.txt','w')
for element in track_ids:
    MyFile.write(str(element))
    MyFile.write('\n')
MyFile.close()

In [9]:
# replace list of genre's with the top genre's id and rename column
genres_formatted = []
for index in range(len(tracks)):
    genres = tracks.iloc[index]["genres_all"]
    if(type(genres) is str):
        genres2 = genres[1:-1]
        if("," in genres):
            genres_formatted=list(genres2.split(", "))
            genres_formatted=[int(x) for x in genres_formatted]
            genre = genres_formatted[0]
        else:
            genre=genres2
    else:
        genre = genres2
 
    tracks.iloc[index]["genres_all"] = genre
    genres_formatted=[]


In [10]:
tracks.head(10)

,track_id,title,title,location,name,bit_rate,genres_all
2,2,AWOL - A Way Of Life,Food,New Jersey,AWOL,256000,21
3,3,AWOL - A Way Of Life,Electric Ave,New Jersey,AWOL,256000,21
4,5,AWOL - A Way Of Life,This World,New Jersey,AWOL,256000,21
5,10,Constant Hitmaker,Freeway,NaN,Kurt Vile,192000,10
6,20,Niris,Spiritual Level,Colchester England,Nicky Cook,256000,17
7,26,Niris,Where is your Love?,Colchester England,Nicky Cook,256000,17
8,30,Niris,Too Happy,Colchester England,Nicky Cook,256000,17
9,46,Niris,Yosemite,Colchester England,Nicky Cook,256000,17
10,48,Niris,Light of Light,Colchester England,Nicky Cook,256000,17
11,134,AWOL - A Way Of Life,Street Music,New Jersey,AWOL,256000,21


In [11]:
#rename genres_all column to genre_id
tracks = tracks.rename(columns={'genres_all': 'genre_id'})
tracks.head(10)

,track_id,title,title,location,name,bit_rate,genre_id
2,2,AWOL - A Way Of Life,Food,New Jersey,AWOL,256000,21
3,3,AWOL - A Way Of Life,Electric Ave,New Jersey,AWOL,256000,21
4,5,AWOL - A Way Of Life,This World,New Jersey,AWOL,256000,21
5,10,Constant Hitmaker,Freeway,NaN,Kurt Vile,192000,10
6,20,Niris,Spiritual Level,Colchester England,Nicky Cook,256000,17
7,26,Niris,Where is your Love?,Colchester England,Nicky Cook,256000,17
8,30,Niris,Too Happy,Colchester England,Nicky Cook,256000,17
9,46,Niris,Yosemite,Colchester England,Nicky Cook,256000,17
10,48,Niris,Light of Light,Colchester England,Nicky Cook,256000,17
11,134,AWOL - A Way Of Life,Street Music,New Jersey,AWOL,256000,21


In [12]:
tracks.shape

(106574, 7)

### Read features

In [13]:
features = pd.read_csv("../Data/features.csv")

In [14]:
# reformatting features data
features = features.drop(1)
features = features.drop(2)
features = features.drop(0)

columnNames = features.columns.values
columnNames[0] = "track_id"
features.columns = columnNames
features.head(10)

,track_id,chroma_cens,chroma_cens.1,chroma_cens.2,chroma_cens.3,chroma_cens.4,chroma_cens.5,chroma_cens.6,chroma_cens.7,chroma_cens.8,...,tonnetz.39,tonnetz.40,tonnetz.41,zcr,zcr.1,zcr.2,zcr.3,zcr.4,zcr.5,zcr.6
3,2,7.1806526184e+00,5.2303090096e+00,2.4932080507e-01,1.3476201296e+00,1.4824777842e+00,5.3137123585e-01,1.4815930128e+00,2.6914546490e+00,8.6686819792e-01,...,5.4125156254e-02,1.2225749902e-02,1.2110591866e-02,5.7588901520e+00,4.5947265625e-01,8.5629448295e-02,7.1289062500e-02,0.0000000000e+00,2.0898721218e+00,6.1448108405e-02
4,3,1.8889633417e+00,7.6053929329e-01,3.4529656172e-01,2.2952005863e+00,1.6540306807e+00,6.7592434585e-02,1.3668476343e+00,1.0540937185e+00,1.0810308903e-01,...,6.3831120729e-02,1.4211839065e-02,1.7740072682e-02,2.8246941566e+00,4.6630859375e-01,8.4578499198e-02,6.3964843750e-02,0.0000000000e+00,1.7167237997e+00,6.9330163300e-02
5,5,5.2756297588e-01,-7.7654317021e-02,-2.7961030602e-01,6.8588310480e-01,1.9375696182e+00,8.8083887100e-01,-9.2319184542e-01,-9.2723226547e-01,6.6661673784e-01,...,4.0730185807e-02,1.2690781616e-02,1.4759079553e-02,6.8084154129e+00,3.7500000000e-01,5.3114086390e-02,4.1503906250e-02,0.0000000000e+00,2.1933031082e+00,4.4860601425e-02
6,10,3.7022454739e+00,-2.9119303823e-01,2.1967420578e+00,-2.3444947600e-01,1.3673638105e+00,9.9841135740e-01,1.7706941366e+00,1.6045658588e+00,5.2121698856e-01,...,7.4357867241e-02,1.7951935530e-02,1.3921394013e-02,2.1434211731e+01,4.5214843750e-01,7.7514506876e-02,7.1777343750e-02,0.0000000000e+00,3.5423245430e+00,4.0800448507e-02
7,20,-1.9383698702e-01,-1.9852678478e-01,2.0154602826e-01,2.5855624676e-01,7.7520370483e-01,8.4794059396e-02,-2.8929358721e-01,-8.1641042233e-01,4.3850939721e-02,...,9.5002755523e-02,2.2492416203e-02,2.1355332807e-02,1.6669036865e+01,4.6972656250e-01,4.7224905342e-02,4.0039062500e-02,9.7656250000e-04,3.1898307800e+00,3.0992921442e-02
8,26,-6.9953453541e-01,-6.8415790796e-01,4.8824872822e-02,4.2658798397e-02,-8.1896692514e-01,-9.1712284088e-01,-9.0183424950e-01,-6.6844828427e-02,-2.9103723168e-01,...,1.0371652246e-01,2.5541320443e-02,2.3846302181e-02,4.1645809174e+01,2.5048828125e-01,1.8387714401e-02,1.5625000000e-02,0.0000000000e+00,4.6905956268e+00,1.4598459937e-02
9,30,-7.2148716450e-01,-8.4855991602e-01,8.9090377092e-01,8.8619679213e-02,-4.4551330805e-01,-1.2711701393e+00,-1.2401897907e+00,-1.3437650204e+00,-9.0560036898e-01,...,1.4169253409e-01,2.0426128060e-02,2.5417611003e-02,8.1665945053e+00,5.4687500000e-01,5.4416511208e-02,3.6132812500e-02,2.4414062500e-03,2.2447082996e+00,5.2673552185e-02
10,46,-1.1970755458e-01,-8.5881441832e-01,2.3625464439e+00,1.0658428818e-01,-1.3159115314e+00,-1.2203541994e+00,-1.6681618690e+00,-5.1603251696e-01,-5.3639507294e-01,...,1.3263167441e-01,3.3211655915e-02,2.1309997886e-02,1.4731082916e+01,2.2363281250e-01,3.6600999534e-02,3.2226562500e-02,0.0000000000e+00,2.8487360477e+00,2.0713411272e-02
11,48,-1.0540534258e+00,9.3233942986e-01,5.2806353569e-01,-1.0353376865e+00,-1.0006815195e+00,-1.1193039417e+00,1.1669902802e+00,-1.0026028156e+00,-1.0949990749e+00,...,1.4195451140e-01,2.4653503671e-02,2.5202710181e-02,2.4550788879e+01,3.7158203125e-01,3.3015336841e-02,2.2460937500e-02,0.0000000000e+00,4.4067325592e+00,3.9016269147e-02
12,134,9.1844475269e-01,6.7414724827e-01,5.7781827450e-01,1.2811170816e+00,9.3374562263e-01,7.8176945448e-02,1.1992042065e+00,-1.7522314191e-01,9.2548191547e-01,...,5.8766160160e-02,1.6322381794e-02,1.5819497406e-02,4.7310867310e+00,4.1943359375e-01,6.4369551837e-02,5.0781250000e-02,0.0000000000e+00,1.8061059713e+00,5.4622855037e-02


In [15]:
features.columns

Index(['track_id', 'chroma_cens', 'chroma_cens.1', 'chroma_cens.2',
       'chroma_cens.3', 'chroma_cens.4', 'chroma_cens.5', 'chroma_cens.6',
       'chroma_cens.7', 'chroma_cens.8',
       ...
       'tonnetz.39', 'tonnetz.40', 'tonnetz.41', 'zcr', 'zcr.1', 'zcr.2',
       'zcr.3', 'zcr.4', 'zcr.5', 'zcr.6'],
      dtype='object', length=519)

In [16]:
features.shape

(106574, 519)

In [17]:
# combine features with tracks by track_id
final_dataset = pd.merge(tracks, features)
final_dataset.shape

(91213, 525)

In [18]:
final_dataset.to_csv(r'..\Data\preprocessed_data.csv', index=False)